In [14]:
import os
import keras
import keras.api.layers as layers
import keras.api.models as models
import keras.api.applications as applications
from keras.api.utils import image_dataset_from_directory
from keras.api.callbacks import EarlyStopping, ModelCheckpoint

os.environ['KERAS_BACKEND'] = 'tensorflow'
os.environ["XLA_PYTHON_CLIENT_MEM_FRACTION"] = "1.0"

train_dataset = image_dataset_from_directory('datasets/images/train', label_mode='categorical', image_size=(224, 224))
val_dataset = image_dataset_from_directory('datasets/images/val', label_mode='categorical', image_size=(224, 224))
test_dataset = image_dataset_from_directory('datasets/images/test', label_mode='categorical', image_size=(224, 224))



Found 9166 files belonging to 8 classes.
Found 611 files belonging to 8 classes.
Found 628 files belonging to 8 classes.


In [25]:
inputs = keras.layers.Input(shape=(224, 224, 3))
backbone = applications.Xception(input_shape=(224, 224, 3), include_top=False, classes=8)
backbone.trainable = False

x = backbone(inputs)
x = layers.GlobalAveragePooling2D()(x)
outputs = layers.Dense(units=8, activation='softmax')(x)

model = keras.Model(inputs=inputs, outputs=outputs)
model.summary()

Model: "functional_8"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_26 (InputLayer)     │ (None, 224, 224, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ xception (Functional)           │ (None, 7, 7, 2048)     │    20,861,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ global_average_pooling2d_2      │ (None, 2048)           │             0 │
│ (GlobalAveragePooling2D)        │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 8)              │        16,392 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,877,872 (79.64 MB)

 Trainable params: 16,392 (64.03 KB)

 Non-trainable params: 20,861,480 (79.58 MB)

In [26]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [27]:
fit_history = model.fit(train_dataset,
                        epochs=1,
                        validation_data=val_dataset,
                        batch_size=32,
                        callbacks=[
                            EarlyStopping(monitor='val_loss', patience=5),
                            ModelCheckpoint('model_output/weights.keras', save_best_only=True)
                        ])

287/287 ━━━━━━━━━━━━━━━━━━━━ 0s 1s/step - accuracy: 0.3772 - loss: 5.3317

287/287 ━━━━━━━━━━━━━━━━━━━━ 420s 1s/step - accuracy: 0.3774 - loss: 5.3261 - val_accuracy: 0.3944 - val_loss: 2.7106
